In [3]:
import torch
from torch import empty , cat , arange
import torch.nn.functional as F
from torch.nn.functional import fold , unfold
from others.convolution import *
from others.activations import *
from others.container import *
from others.loss import *


In [111]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [112]:
%reload_ext autoreload

In [4]:
torch.set_grad_enabled(True)

In [85]:
def test_loss(loss, ref_loss):
    for loop in range(10):
        x = torch.randn((2 , 3 , 32 , 32), requires_grad=True)
        y = torch.randn((2 , 3 , 32 , 32))
        l1 = loss(x,y)
        l2 = ref_loss(x,y) 
    
        torch.testing.assert_allclose( l1, l2)
        
        grad_1 = loss.backward()
        l2.backward()
        grad_2 = x.grad
    
        torch.testing.assert_allclose(grad_1, grad_2)
    
    x = torch.randn((1 , 3 , 32 , 32),requires_grad=True)

In [86]:
test_loss(MSE(),F.mse_loss)

In [87]:
def test_activations(act, ref_act):
    mse = MSE()
    for loop in range(10):
        x = torch.randn((2 , 1 , 3 , 3), requires_grad=True)
        y = torch.randn((2 , 1 , 3 , 3))
        
        x1 = act(x)
        x2 = ref_act(x)
        
        l1 = mse(x1,y)
        l2 = F.mse_loss(x2,y)
        torch.testing.assert_allclose( l1, l2)
        
        grad_1 = act.backward(mse.backward())
        l2.backward()
        grad_2 = x.grad
    
        torch.testing.assert_allclose(grad_1, grad_2)
        
    
    
    x = torch.randn((1 , 3 , 32 , 32),requires_grad=True)

In [88]:
test_activations(ReLU(), F.relu)

In [84]:
test_activations(Sigmoid(), torch.sigmoid)

In [80]:
def test_block(block, ref_block):
    mse = MSE()
    for loop in range(10):
        x = torch.randn((2 , 3 , 32 , 32), requires_grad=True)
        y = torch.randn((2 , 3 , 32 , 32))
        
        x1 = block(x)
        x2 = ref_block(x)
        torch.testing.assert_allclose( x1, x2)
        
        y = torch.randn(x1.shape)
        
        l1 = mse(x1,y)
        l2 = F.mse_loss(x2,y)
        torch.testing.assert_allclose( l1, l2)
        
        grad_1 = block.backward(mse.backward())
        l2.backward()
        grad_2 = x.grad
    
        torch.testing.assert_allclose(grad_1, grad_2)
      
        
    
     
    
   
   
 

In [81]:
in_channels = 3
out_channels=3
kernel_size=(2, 2)
stride=1

conv = Conv2d(in_channels=in_channels, out_channels=out_channels, kernel_size=kernel_size, stride=stride)
conv_ref = lambda x : F.conv2d(x, conv.weight, conv.bias) 
test_block(conv, conv_ref)

In [113]:
in_channels = 3
out_channels=3
kernel_size=(2, 2)
stride=1


trans_conv = Upsampling(in_channels, out_channels, kernel_size, stride=stride)
trans_conv_ref = lambda x : F.conv_transpose2d(x, trans_conv.weight, bias=trans_conv.bias)
test_block(trans_conv, trans_conv_ref)

AttributeError: 'TransposeConv2d' object has no attribute 'bais'

In [107]:
Upsampling??

Init signature:
Upsampling(
    in_channels=3,
    out_channels=3,
    kernel_size=(2, 2),
    stride=1,
    padding=0,
    dilation=1,
)
Docstring:      Upsampling module (underlying is TransposeConv2d)
Init docstring: Upsampling constructor
File:           ~/Bureau/noise2noise/Proj/Miniproject_2/others/convolution.py
Type:           type
Subclasses:     
